In [13]:
from tensorflow import keras
import sys
sys.path.append('..')
from utils import load_meters_data
from models.autoencoder import prepare_data
model = keras.models.load_model('../models/autoencoder1.0')

ModuleNotFoundError: No module named 'autoencoder'

In [9]:
meters_data = load_meters_data()
X_train, X_test = prepare_data(meters_data, time_steps=10, split_ratio=0)

FileNotFoundError: [Errno 2] No such file or directory: 'data/EsaveExport_Trondheim Kommune_Trondheim_10121314.xls'